# Custom Object and Scene detection using Amazon Rekognition Custom Labels API

***
This notebook provides a walkthrough of [Amazon Rekognition Custom Labels API](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/custom-labels-api-reference.html) to identify custom objects and scenes.
***

***
<b>Setup</b>
1. Amazon SageMaker Notebook is not a requirement to use Amazon Rekognition APIs. For this notebook, it provides an environment to run python code and make API calls.
2. Create an [Amazon SageMaker Notebook instance](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html)
3. Click the Amazon SageMaker Notebook instance and open the Role.
4. For this demonstration in the Permissions tab, choose Attach policies. Add the AmazonRekognitionCustomLabelsFullAccess, AmazonSageMakerFullAccess and AmazonS3FullAccess. This provides full access to Amazon Sagemaker,Amazon Rekognition Custom labels and Amazon S3 service but we would recommend to trim down the access policies for a production application.
5. Return to the Amazon Sagemaker Notebook insatnce, Under Actions, click on “Open JupyterLab”
6. Open the terminal
7. Run cd SageMaker/
8. Run git clone https://github.com/aws-samples/amazon-rekognition-code-samples.git
9. Open the “custom-labels” folder within “amazon-rekognition-code-samples” created under “Files” tab.
10. You should see two jupyter notebooks. You can continue with the rest of the notebook.
11. If you receive Kernel not found, select “conda_python3” from the dropdown on Top right.
***

In [ ]:
# ## Upgrade boto3 and botocore
# !pip install botocore --upgrade
# !pip install boto3 --upgrade

### Initialize Notebook

In [ ]:
import json
import xml.etree.ElementTree as ET
from datetime import datetime
from os import listdir, makedirs
from os.path import isfile, join
import shutil

import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

from IPython.display import JSON

In [ ]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = mySession.region_name
print ("AWS Region : " + awsRegion)

In [ ]:
# Init clients
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

***
Activities
***
In this notebook, we will Train an Amazon Rekognition Custom Label model to detect different Dog breeds.
1. Initialize Amazon Rekognition Custom Labels.
2. Download the Public Stanford Dog Datasets.
3. Prepare the datasets and generating manifest file.
4. Create an Amazon Rekognition Custom Label Project.
5. Import the Datasets into the new Amazon Rekognition Custom Label Project.
6. Split the dataset into Training and Test using Rekognition Custom Label API.
7. Train a Custom Model to detect Dog breed.
8. Retrieve the Model Metrics.
9. Start the Inference endpoint.
10. Run predictions on the holdout dataset.
11. Stop the Inference endpoint.
***

### Initialize Amazon Rekognition Custom Labels

- 아래에 bucket 명은 AWS console에서 Rekognition custom label 들어가서 만든 기본 bucket 이름을 입력합니다.

In [ ]:
## Please replace the <rek_cl_default_bucket_name> with the value of the Amazon S3 bucket name captured above. It will be different in your case.
rek_cl_default_bucket = "<rek_cl_default_bucket_name>"

### Download the Public Stanford Dog Datasets
***
1. Download the images in the dataset from Stanford University.
2. Download the annotation of the images in the dataset from Stanford University.
3. Prepare the datsets and image annotation using the manifest file.
4. Copy the dataset to Rekognition Custom Label's default bucket.
***

In [ ]:
## Download the images in the dataset from Stanford University found here - http://vision.stanford.edu/aditya86/ImageNetDogs/.
## Please note you comply with all license requirements before downloading the datasets.
## You may manually download the dataset and place it in the respective directories in the your current directory.
## The below sample code snippet provides guidance around the steps for downloading the dataset .

## Dataset and Annotation dataset url
stanford_dog_dataset_image_url = "http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar"
stanford_dog_dataset_annotation_url = "http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar"

## Creating directories in the current location
!mkdir -p ./stanford_dog_dataset/images_tar
!mkdir -p ./stanford_dog_dataset/images
!mkdir -p ./stanford_dog_dataset/annotation_tar
!mkdir -p ./stanford_dog_dataset/annotation

## Download the dataset to the directories created above
!curl $stanford_dog_dataset_image_url --output ./stanford_dog_dataset/images_tar/images.tar
!curl $stanford_dog_dataset_annotation_url --output ./stanford_dog_dataset/annotation_tar/annotation.tar


## Unzipping the downlaoded datasets.
!tar -xf ./stanford_dog_dataset/images_tar/images.tar -C ./stanford_dog_dataset/images --no-same-owner
!tar -xf ./stanford_dog_dataset/annotation_tar/annotation.tar -C ./stanford_dog_dataset/annotation --no-same-owner

### Prepare the datasets and generating manifest file
***
The original dataset contains 120 classes and 20,000+ images. 
<br>This is possible to train with custom labels, but it will take many hours. 
<br>In order to reduce the training time we will trim the classes to 3 and give them easier to read names. 
<br>We will also demonstrate the generation of a Amazon Sageamker Groundtruth manifest file from the XML format annotation available with the public dataset. 
<br>We will also select 1 image from each class along with an image from unknown class and keep it aside as a holdout dataset.
***

In [ ]:
## Function to convert a list of stanford dog dataset annotation to generate a single Amazon Sagemaker groundtruth manifest file.
def generate_manifest_file (stanford_dog_dataset_annotation, final_manifest_file_path):
    final_file=[]
    class_dict={}
    for filename in stanford_dog_dataset_annotation:
        file_pth = filename.replace('/images/Images/','/annotation/Annotation/').replace('.jpg','')
        # print(file_pth)
        file = open(file_pth, mode = 'r' )
        annt = file.read()
        # print (annt)
        singl_file = {}
        # print ("\n")
        root = ET.fromstring(annt)
        img_size = {}
        img_nm = None
        for child in root:
            if (child.tag == 'filename'):
                # print (child.text)
                img_nm = child.text

            if (child.tag == 'size'):
                for elem in child.iter():
                    if (elem.tag == 'width'):
                        # print (elem.text)
                        img_size["width"]=int(elem.text)
                    if (elem.tag == 'height'):
                        # print (elem.text)
                        img_size["height"] = int(elem.text)
                    if (elem.tag == 'depth'):
                        # print (elem.text)
                        img_size["depth"] = int(elem.text)
        # print (json.dumps(singl_file))
        annotations = []
        objects = []
        class_map = {}
        objcts = root.findall('object')
        # print (len(objcts))
        for objct in objcts:
            annotation = {}
            confidence = {}
            class_nm = None
            class_id = None
            for elem in objct.iter():
                if (elem.tag == 'name'):
                    # print(elem.text)
                    class_nm = elem.text
                    class_id = class_dict.get(class_nm, None)
                    if class_id == None:
                        if len(class_dict) == 0:
                            class_id = 0
                        else:
                            all_values = class_dict.values()
                            max_value = max(all_values)
                            class_id = int(max_value) + 1
                        class_dict[class_nm] = class_id

                if (elem.tag == 'bndbox'):
                    xmin = ymin = xmax = ymax = 0
                    for subelem in elem.iter():
                        if (subelem.tag == 'xmin'):
                            # print(subelem.text)
                            xmin = int(subelem.text)
                        if (subelem.tag == 'ymin'):
                            # print(subelem.text)
                            ymin = int(subelem.text)
                        if (subelem.tag == 'xmax'):
                            # print(subelem.text)
                            xmax = int(subelem.text)
                        if (subelem.tag == 'ymax'):
                            # print(subelem.text)
                            ymax = int(subelem.text)

                    annotation["class_id"] = class_id
                    annotation["top"] = ymin
                    annotation["left"] = xmin
                    annotation["width"] = xmax - xmin
                    annotation["height"] = ymax - ymin
                    annotations.append(annotation)

                    confidence["confidence"] = 1
                    objects.append(confidence)

                    class_map[class_id] = class_nm
        bbx = {
            "image_size" : [img_size],
            "annotations" : annotations
        }

        bbx_mtdata = {
            "objects" : objects,
            "class-map" : class_map,
            "type": "groundtruth/object-detection",
            "human-annotated": "yes",
            "creation-date": datetime.today().strftime('%Y-%m-%dT%H:%m:%S'), 
            "job-name": "testjob"
        }


        singl_file = {
            "source-ref" : f's3://{rek_cl_default_bucket}/stanford_dog_dataset/dataset/{img_nm}.jpg',
            "bounding-box" : bbx,
            "bounding-box-metadata" : bbx_mtdata

        }

        # print (json.dumps(singl_file))
        imglbl = json.dumps(singl_file)
        final_file.append(imglbl + " \n")
        file.close()
    
    file1 = open(final_manifest_file_path, "w")
    file1.writelines(final_file)
    file1.close()
    
    print(f"Local manifest path : {final_manifest_file_path}")

In [ ]:
classes_to_be_trained = [
    "n02090379-redbone", 
    "n02099601-golden_retriever", 
    "n02107142-Doberman" 
]

holdoutset_list = {
    "n02090379-redbone" : "./stanford_dog_dataset/images/Images/n02090379-redbone/n02090379_1799.jpg",
    "n02099601-golden_retriever" : "./stanford_dog_dataset/images/Images/n02099601-golden_retriever/n02099601_3388.jpg",
    "n02107142-Doberman" : "./stanford_dog_dataset/images/Images/n02107142-Doberman/n02107142_385.jpg",
    ## the below image is from a new class which is not included as part of the training set
    "n02091831-Saluki" : "./stanford_dog_dataset/images/Images/n02091831-Saluki/n02091831_3909.jpg"
}

In [ ]:
dataset_list = []

## Generating trim dataset
makedirs("./stanford_dog_dataset/holdout", exist_ok=True)
for class_nm in classes_to_be_trained:
    new_path = f'./stanford_dog_dataset/dataset/'
    makedirs(new_path, exist_ok=True)
    
    existing_path = f'./stanford_dog_dataset/images/Images/{class_nm}'
    onlyfiles = [join(existing_path, f) for f in listdir(existing_path) if isfile(join(existing_path, f))]
    # remove the image from the list which is kept as part of the holdout dataset.
    onlyfiles.remove(holdoutset_list[class_nm])
    
    for dataset_file_pth in onlyfiles:
        dataset_list.append(dataset_file_pth)
        dest_dataset_image_path = f'{new_path}/{dataset_file_pth.split("/")[-1]}'
        shutil.copy(dataset_file_pth, dest_dataset_image_path)

## Generating dataset manifest file
generate_manifest_file (dataset_list, './stanford_dog_dataset/dataset/dataset.manifest')



### Custom label을 위한 정답 포맷
- 위의 과정은 custom dataset의 정답을 Amazon rekognition에서 활용할 수 있는 형태로 변경하는 과정입니다. 아래와 같은 포맷을 가지게 됩니다.

```
{
   "source-ref":"s3://custom-labels-console-ap-northeast-2-feed27152b/stanford_dog_dataset/dataset/n02090379_4611.jpg",
   "bounding-box":{
      "image_size":[
         {
            "width":500,
            "height":338,
            "depth":3
         }
      ],
      "annotations":[
         {
            "class_id":0,
            "top":11,
            "left":90,
            "width":261,
            "height":326
         }
      ]
   },
   "bounding-box-metadata":{
      "objects":[
         {
            "confidence":1
         }
      ],
      "class-map":{
         "0":"redbone"
      },
      "type":"groundtruth/object-detection",
      "human-annotated":"yes",
      "creation-date":"2023-05-15T07:05:22",
      "job-name":"testjob"
   }
}
```

- AWS 문서 참고: https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/md-dataset-purpose.html#md-dataset-purpose-localization
- 예시 참고: https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/md-localize-objects.html

In [ ]:

## Copying the holdout dataset images
for class_nm, file_path in holdoutset_list.items():
    dest_holdout_image_path = f'./stanford_dog_dataset/holdout/{class_nm.split("-")[-1]}.jpg'
    shutil.copy(file_path, dest_holdout_image_path)
    
    
    

## Copy the datasets and holdout set to the Amazon Rekognition Custom Label default S3 bucket
!aws s3 cp ./stanford_dog_dataset/dataset/ s3://$rek_cl_default_bucket/stanford_dog_dataset/dataset/ --recursive --quiet
!aws s3 cp ./stanford_dog_dataset/holdout/ s3://$rek_cl_default_bucket/stanford_dog_dataset/holdout/ --recursive --quiet
    


### Create an Amazon Rekognition Custom Label Project
***
Create an Amazon Rekognition Custom Label Project using the APIs
***

In [ ]:
cl_project = rekognition.create_project(
    ProjectName='stanford_dogs_rek_custom'
)
JSON(cl_project)

### Import the Datasets into the new Amazon Rekognition Custom Label Project
***
Import the Datasets into the new Amazon Rekognition Custom Label Project using the APIs.
In this section we will import the entire dataset as training dataset and later we will demonstrate how we can use the Amazon Rekgniton out of the box to automatically keep aside 20% of the dataset as Test dataset.

Training Dataset -> A set of labeled or annotated images which will be used to train the model. The model will be trained on this dataset.
<br>
Test Dataset -> A set of labeled or annotated images which will be used to evaluate the model.
***

In [ ]:
## We will use the dataset.manifest file which was created earlier, to create a training dataset.
cl_dataset_train = rekognition.create_dataset(
                        DatasetSource={
                            'GroundTruthManifest': {
                                'S3Object': {
                                    'Bucket': rek_cl_default_bucket,
                                    'Name': 'stanford_dog_dataset/dataset/dataset.manifest'
                                }
                            }
                        },
                        DatasetType='TRAIN',
                        ProjectArn=cl_project['ProjectArn']
                    )
JSON(cl_dataset_train)

In [ ]:
## Wait for the creation of the train dataset to complete
chk_status = True
starttime = time.time()
while (chk_status):
    ## wait for 1 minute. To check status every 1 minutes
    time.sleep (60)
    dataset_status = rekognition.describe_dataset(
                            DatasetArn=cl_dataset_train['DatasetArn']
                        )
    if ( (dataset_status['DatasetDescription']['Status'] != 'CREATE_IN_PROGRESS') ):
        chk_status = False
    ## Continue to check for status for 1 hour
    if ((time.time() - starttime) > 3600):
        chk_status = False
        # Raise an exception if needed
        
JSON(dataset_status)

In [ ]:
## We will a create an empty test dataset. As we will use the distribute dataset api later to keep aside 20% of training data for the training dataset
cl_dataset_test = rekognition.create_dataset(
                DatasetType='TEST',
                ProjectArn=cl_project['ProjectArn']
            )
JSON(cl_dataset_test)

In [ ]:
## Wait for the creation of the empty test dataset to complete
chk_status = True
starttime = time.time()
while (chk_status):
    ## wait for 1 minute. To check status every 1 minutes
    time.sleep (60)
    dataset_status = rekognition.describe_dataset(
                            DatasetArn=cl_dataset_test['DatasetArn']
                        )
    if ( (dataset_status['DatasetDescription']['Status'] != 'CREATE_IN_PROGRESS') ):
        chk_status = False
    ## Continue to check for status for 1 hour
    if ((time.time() - starttime) > 3600):
        chk_status = False
        # Raise an exception if needed
        
JSON(dataset_status)

### Split the dataset into Training and Test datasets using Rekognition Custom Label API
***
Here we will use the [distribute_dataset_entries](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.distribute_dataset_entries) API to automatically generate a Test dataset which will be used to evaluate the model.
<br>The activity of splitting the dataset needs to be done only once for each project. Later we can add images to the training or test dataset using the [update_dataset_entries](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.update_dataset_entries) api to update/extend the training and test datasets.
<br>Another option would be to create a training and test dataset. Then use [create_dataset](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.create_dataset) api to create a training and test dataset for that project. </br>
***

In [ ]:
## Split the dataset that was created earlier into Training and Test dataset
cl_distribute_dataset = rekognition.distribute_dataset_entries(
                            Datasets=[
                                {
                                    'Arn': cl_dataset_train['DatasetArn']
                                },
                                {
                                    'Arn': cl_dataset_test['DatasetArn']
                                }
                            ]
                        )


In [ ]:
## Wait for the splitting of the dataset to complete
chk_status = True
starttime = time.time()
while (chk_status):
    ## wait for 1 minute. To check status every 1 minutes
    time.sleep (60)
    dataset_status_train = rekognition.describe_dataset(
                                DatasetArn=cl_dataset_train['DatasetArn']
                            )
    dataset_status_test = rekognition.describe_dataset(
                                DatasetArn=cl_dataset_test['DatasetArn']
                            )
    if ( (dataset_status_train['DatasetDescription']['Status'] != 'CREATE_IN_PROGRESS') and (dataset_status_test['DatasetDescription']['Status'] != 'CREATE_IN_PROGRESS') ):
        chk_status = False
    ## Continue to check for status for 1 hour
    if ((time.time() - starttime) > 3600):
        chk_status = False
        # Raise an exception if needed
        
JSON(dataset_status_train)


In [ ]:
JSON(dataset_status_test)

Check datasets from the AWS Console if needed

### Train a Custom Model to detect Dog breed
***
Here we will use the [create_project_version](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.create_project_version) API to train the model.
***

In [ ]:
## Start Training the model
model_version_name = f'model_v{str(int(time.time()))}'
cl_train_model = rekognition.create_project_version(
                        ProjectArn=cl_project['ProjectArn'],
                        VersionName=model_version_name,
                        OutputConfig={
                            'S3Bucket': rek_cl_default_bucket,
                            'S3KeyPrefix': 'stanford_dog_dataset/model_train'
                        }
                    )
JSON(cl_train_model)

In [ ]:
## Wait for the training to finish. This may take 2 to 4 hours
chk_status = True
starttime = time.time()
while (chk_status):
    ## wait for 2 minute. To check status every 2 minutes
    time.sleep (120)
    model_traing_status = rekognition.describe_project_versions(
                                ProjectArn=cl_project['ProjectArn'],
                                VersionNames=[
                                               model_version_name
                                            ]
        
                            )
    if ( (model_traing_status['ProjectVersionDescriptions'][0]['Status'] != 'TRAINING_IN_PROGRESS') ):
        chk_status = False
    ## Continue to check for status for 10 hour
    if ((time.time() - starttime) > 36000):
        chk_status = False
        # Raise an exception if needed
    print("Training ...")
        
print("Finished. Check the status")
JSON(model_traing_status)


### Retrieve the Model Metrics
***
Here we will use the [describe_project_versions](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.describe_project_versions) API to retrive the trained model metrics.
<br>To understand mroe about the metrics please refer [Metrics for evaluating your model](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/im-metrics-use.html)
<br> To programatically access more granular results related to the model. Please refer [Accessing Amazon Rekognition Custom Labels training results](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/im-metrics-api.html)
***

In [ ]:
model_metrics = rekognition.describe_project_versions(
                                ProjectArn=cl_project['ProjectArn'],
                                VersionNames=[
                                               model_version_name
                                            ]
        
                            )
# model_metrics
print ("F1 Score " + str(model_metrics['ProjectVersionDescriptions'][0]['EvaluationResult']['F1Score']))

s3 = boto3.resource('s3')
content_object = s3.Object(model_metrics['ProjectVersionDescriptions'][0]['EvaluationResult']['Summary']['S3Object']['Bucket'], 
                          model_metrics['ProjectVersionDescriptions'][0]['EvaluationResult']['Summary']['S3Object']['Name'] )
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)

JSON(json_content)

### 학습 지표 확인

- 학습 완료된 내용은 AWS console에서도 동일하게 확인이 가능합니다.

### Start the trained model
***
Here we will use the [start_project_version](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.start_project_version) API to start the trained model.
***

In [ ]:
start_model = rekognition.start_project_version(
                        ProjectVersionArn=model_metrics['ProjectVersionDescriptions'][0]['ProjectVersionArn'],
                        MinInferenceUnits=1
                    )
JSON (start_model)

In [ ]:
## Wait for the model to start
chk_status = True
starttime = time.time()
while (chk_status):
    ## wait for 1 minute. To check status every 1 minutes
    time.sleep (60)
    model_start_status = rekognition.describe_project_versions(
                                ProjectArn=cl_project['ProjectArn'],
                                VersionNames=[
                                               model_version_name
                                            ]
        
                            )
    if ( (model_start_status['ProjectVersionDescriptions'][0]['Status'] != 'STARTING') ):
        chk_status = False
    ## Continue to check for status for 1 hour
    if ((time.time() - starttime) > 3600):
        chk_status = False
        # Raise an exception if needed
        
JSON(model_start_status)


### Run tests on the holdout images
***
Here we will use the [detect_custom_labels](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.detect_custom_labels) API to retrieve the dog breeds. These images were kept aside and was not included into the training or test datasets.
***

In [ ]:
#Copyright 2021 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-custom-labels-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
import io
import logging
from PIL import Image, ImageDraw, ImageFont
from IPython.display import HTML, display


from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)

def analyze_local_image(rek_client, model, photo, min_confidence):
    """
    Analyzes an image stored as a local file.
    :param rek_client: The Amazon Rekognition Boto3 client.
    :param s3_connection: The Amazon S3 Boto3 S3 connection object.
    :param model: The ARN of the Amazon Rekognition Custom Labels model that you want to use.
    :param photo: The name and file path of the photo that you want to analyze.
    :param min_confidence: The desired threshold/confidence for the call.
    """

    try:
        logger.info ("Analyzing local file: %s", photo)
        image=Image.open(photo) 
        image_type=Image.MIME[image.format]

        if (image_type == "image/jpeg" or image_type== "image/png") == False:
            logger.error("Invalid image type for %s", photo)
            raise ValueError(
                f"Invalid file format. Supply a jpeg or png format file: {photo}"
            )
            
        # get images bytes for call to detect_anomalies
        image_bytes = io.BytesIO()
        image.save(image_bytes, format=image.format)
        image_bytes = image_bytes.getvalue()

        response = rek_client.detect_custom_labels(Image={'Bytes': image_bytes},
            MinConfidence=min_confidence,
            ProjectVersionArn=model)

        show_image (image, response)
        return len(response['CustomLabels'])

    except ClientError as err:
        logger.error(format(err))
        raise
    

def analyze_s3_image(rek_client,s3_connection, model,bucket,photo, min_confidence):
    """
    Analyzes an image stored in the specified S3 bucket.
    :param rek_client: The Amazon Rekognition Boto3 client.
    :param s3_connection: The Amazon S3 Boto3 S3 connection object.
    :param model: The ARN of the Amazon Rekognition Custom Labels model that you want to use.
    :param bucket: The name of the S3 bucket that contains the image that you want to analyze.
    :param photo: The name of the photo that you want to analyze.
    :param min_confidence: The desired threshold/confidence for the call.
    """

    try:
        #Get image from S3 bucket.
        
        logger.info("analyzing bucket: %s image: %s", bucket, photo)
        s3_object = s3_connection.Object(bucket,photo)
        s3_response = s3_object.get()
        

        stream = io.BytesIO(s3_response['Body'].read())
        image=Image.open(stream)

        image_type=Image.MIME[image.format]

        if (image_type == "image/jpeg" or image_type== "image/png") == False:
            logger.error("Invalid image type for %s", photo)
            raise ValueError(
                f"Invalid file format. Supply a jpeg or png format file: {photo}")
                

        img_width, img_height = image.size  
        draw = ImageDraw.Draw(image)  
        
        #Call DetectCustomLabels 
        response = rek_client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
            MinConfidence=min_confidence,
            ProjectVersionArn=model)

        show_image (image, response)
        return len(response['CustomLabels'])

    except ClientError as err:
        logger.error(format(err))
        raise

def show_image(image, response):
    """
    Displays the analyzed image and overlays analysis results
    :param image: The analyzed image
    :param response: the response from DetectCustomLabels
    """
    try: 
        font_size=10
        line_width=5

        img_width, img_height = image.size  
        draw = ImageDraw.Draw(image)  
                
        # calculate and display bounding boxes for each detected custom label       
        image_level_label_height = 0
        
        for custom_label in response['CustomLabels']:
            confidence=int(round(custom_label['Confidence'],0))
            label_text=f"{custom_label['Name']}:{confidence}%"
#             font = ImageFont.load("arial.pil")
            fnt = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", font_size)
            text_width, text_height = draw.textsize(label_text,fnt)

            logger.info(f"Label: {custom_label['Name']}") 
            logger.info(f"Confidence:  {confidence}%")

            # Draw bounding boxes, if present
            if 'Geometry' in custom_label:
                box = custom_label['Geometry']['BoundingBox']
                left = img_width * box['Left']
                top = img_height * box['Top']
                width = img_width * box['Width']
                height = img_height * box['Height']
                
                logger.info("Bounding box")
                logger.info("\tLeft: {0:.0f}".format(left))
                logger.info("\tTop: {0:.0f}".format(top))
                logger.info("\tLabel Width: {0:.0f}".format(width))
                logger.info("\tLabel Height: {0:.0f}".format(height))
                
                points = (
                    (left,top),
                    (left + width, top),
                    (left + width, top + height),
                    (left , top + height),
                    (left, top))
                #Draw bounding box and label text
                draw.line(points, fill="limegreen", width=line_width)
                draw.rectangle([(left + line_width , top+line_width), (left + text_width + line_width, top + line_width + text_height)],fill="black")
                draw.text((left + line_width ,top +line_width), label_text, fill="limegreen", font=fnt) 
            
            #draw image-level label text.
            else:
                draw.rectangle([(10 , image_level_label_height), (text_width + 10, image_level_label_height+text_height)],fill="black")
                draw.text((10,image_level_label_height), label_text, fill="limegreen", font=fnt)  

                image_level_label_height += text_height
            
#         image.show()
        display(image)

    except Exception as err:
        logger.error(format(err))
        raise
        

def analyze_rekcl_bb(bckt_nm, pics, modelarn, minconfdnce):

    try:
        logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

        bucket=bckt_nm
        photos=pics
        model=modelarn
        min_confidence=minconfdnce
       

        rek_client=boto3.client('rekognition')
        
        s3_connection = boto3.resource('s3')
        for photo in photos:
            label_count=analyze_s3_image(rek_client,
                s3_connection,
                model,
                bucket,
                photo,
                min_confidence)

        """
        # Uncomment to analyze a local file. 
        # Change photo to the path and file name of a local file.
        label_count=analyze_local_image(rek_client,
            model,
            photo,
            min_confidence)
        
        """ 
        logger.info(f"Custom labels detected: {label_count}")

    except ClientError as err:
        print("A service client error occurred: " + format(err.response["Error"]["Message"]))
    except ValueError as err:
        print ("A value error occurred: " + format(err))




In [ ]:
## Test the model on the holdout dataset
bucket_nm=rek_cl_default_bucket
holdout_photos=[('stanford_dog_dataset/holdout/' + f) for f in listdir('./stanford_dog_dataset/holdout') if (isfile(join('./stanford_dog_dataset/holdout', f)) and ((f.split(".")[-1]) == 'jpg' ) )]
model_arn=model_metrics['ProjectVersionDescriptions'][0]['ProjectVersionArn']
min_confidence=70

analyze_rekcl_bb(bucket_nm, holdout_photos, model_arn, min_confidence)


### Stop the trained model
***
Here we will use the [stop_project_version](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.stop_project_version) API to retrive the trained model metrics.
***

In [ ]:
stop_model = rekognition.stop_project_version(
                        ProjectVersionArn=model_metrics['ProjectVersionDescriptions'][0]['ProjectVersionArn']
                    )
JSON (stop_model)

In [ ]:
## Wait for the model to stop
chk_status = True
starttime = time.time()
while (chk_status):
    ## wait for 1 minute. To check status every 1 minutes
    time.sleep (60)
    model_stop_status = rekognition.describe_project_versions(
                                ProjectArn=cl_project['ProjectArn'],
                                VersionNames=[
                                               model_version_name
                                            ]
        
                            )
    if ( (model_stop_status['ProjectVersionDescriptions'][0]['Status'] != 'STOPPING') ):
        chk_status = False
    ## Continue to check for status for 1 hour
    if ((time.time() - starttime) > 3600):
        chk_status = False
        # Raise an exception if needed
        
JSON(model_stop_status)


You have successfully used Amazon Rekognition Custom Label APIs to run a end to end Rekognition Custom Label Training and Inference flows to detect dog breeds.
<br>You must have noticed that the image Saluki.jpg (this is an image of a dog which breed was not included in the training dataset) is incorrectly classified most probably as golden retriever becasue they look similar.
<br>For the model to detect Saluki dog breed, we will continue with this project to add Saluki dog breed images to the datasets and retrigger training. By providing more images related to Saluki dog breed would help the model differentiate between a Saluki and Golden retriver dog breed. Refer the notebook [Add images to the datasets and retrigger a training job using Rekognition Custom Label APIs](add-images-to-datasets-and-retrigger-training-job-using-rekognition-custom-labels-api.ipynb) notebook in the same directory
<br>The apis can be leveraged to integrate with your current system or your existing ML/AI flows.


### Learn more about different Rekognition Custom Labels APIs
***
To learn more about the Rekognition Custom Labels APIs [Amazon Rekognition Custom Labels API reference](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/custom-labels-api-reference.html)

Also refer the [Add images to the datasets and retrigger a training job using Rekognition Custom Label APIs](add-images-to-datasets-and-retrigger-training-job-using-rekognition-custom-labels-api.ipynb) notebook in the same directory.
***

Stop the Amazon Sagemaker Notebook instance, if you created one to run the demonstration